# 多进程通信


## Queue通信  


Queue（[maxsize]）:建立一个共享的队列(其实并不是共享的，实际是克隆的，内部维护着数据的共享)，多个进程可以向队列里存/取数据。其中，参数是队列最大项数，省略则无限制。  
queue队列内置了锁原语，所以也是进程安全的  

  


# 进程池


## 1、进程池定义：

进程池：可以提供指定数量的进程给用户使用,即当有新的请求提交到进程池中时，如果池未满，则会创建一个新的进程用来执行该请求;反之，如果池中的进程数已经达到规定最大值，那么该请求就会等待，只要池中有进程空闲下来，该请求就能得到执行。

## 2、原理图
绿色：子进程没有工作，处于空闲状态（子进程没有调用任务函数）  

粉色：子进程处于工作状态（子进程调用了任务函数）

![alt 属性文本](https://img-blog.csdnimg.cn/02821ca3a3784375be70f42a5eba84b7.png?x-oss-process=image/watermark,type_ZHJvaWRzYW5zZmFsbGJhY2s,shadow_50,text_Q1NETiBAWVpMNDA1MTQxMzE=,size_20,color_FFFFFF,t_70,g_se,x_16)

![alt 属性文本](https://img-blog.csdnimg.cn/c110f435cc96439dbb7fc6d9d35fd829.png?x-oss-process=image/watermark,type_ZHJvaWRzYW5zZmFsbGJhY2s,shadow_50,text_Q1NETiBAWVpMNDA1MTQxMzE=,size_20,color_FFFFFF,t_70,g_se,x_16)

## 3.优点

（1）.提高效率，手动创建线程效率低，节省开辟进程和开辟内存空间的时间及销毁进程的时间  

（2）.节省内存空间，进程所占用的资源比线程大得多，


## 4.注意事项
使用进程池时要用Manager里面的队列Queue()，直接使用pool的队列有可能报错，实际操作中直接跳过子进程的运行  

进程池的大小小于等于CPU的核数

# 信号量Semaphore

互斥锁同时只允许一个进程修改数据，而信号量则允许多个人同时修改数据。互斥锁就相当于你家的卫生间，只有一个坑位，同时只能一个人上厕所；而信号量则相当于公共厕所，有多个坑位，可以多个人同时上厕所。比如现在有10个人需要上厕所，现在公共厕所只有3个坑位，这时候就先有3个人上厕所，等其中一个或者多个人出来之后，另外的人才能进去。

**Semaphore管理一个内置的计数器，每当调用acquire()时内置计数器-1；调用release() 时内置计数器+1；计数器不能小于0；当计数器为0时，acquire()将阻塞进程直到其他进程调用release()**

信号量与进程池、线程池的区别，信号量有锁，而进程池，线程池，没有锁，

**多进程和多线程对比：**
+ 由于python有GIL锁，而且是一个进程管理着1把GIL锁，所以多线程无法使用多核，即同一时刻只能一个线程在运行。
而多进程编程可以使用多核。意味着，多进程可以并行，同一时刻多进程可以使用多个CPU从而同时运行，而多线程不能做到真正的同时运行。

一句话：python中的多线程只能并发不能并行，多进程则可以并行（前提是你的电脑是多核的）

+ 由于多线程只能并发不能并行，所以适合多IO操作，少CPU操作的任务，例如爬虫，磁盘读写等任务。
多进程相反，适合少IO操作，多CPU操作的任务，例如纯计算的任务。
原因是IO操作会存在等待，线程可以在等待的时候让出CPU给其他线程工作，以达到让整个程序一直都在干活不闲着。
但是如果是耗CPU的操作，则几乎不存在等待阻塞的情况，每一个线程不会因为阻塞而让出CPU，而是因为时间片用完而让出CPU。这种情况下，多线程会比单线程更慢，因为多了线程间的切换。  


而对于多进程而言，由于可以并行，多个进程可以同时完成多个耗CPU的操作，节省时间。举个例子，我现在想完成两个比较庞大的运算：A和B任务。就是就可以生成两个子进程，一个计算A任务，一个计算B任务。A和B任务是并行的，比并发更快。但是有种理解是错误的：认为多进程可以一起完成任务A的运算，让A运算的时间减半，这是不行的。
而对于IO操作，当遇到等待的时候，进程会将CPU让出给其他程序的进程，这段期间这个进程还是什么都做不了。而且多进程的切换耗时更多，进程消耗的资源更多（如内存，fork一个进程是要拷贝一份父进程的内存的），因此还不如用多线程。

+ 多线程的切换比多进程的切换的损耗要少很多，多进程的切换更慢。线程是轻量级的，进程是重量级的。